#### 1 - Obtenção de dados

In [1]:
import requests as req
import json
import time
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
# URL base da API
base_url = 'https://dadosabertos.aneel.gov.br/api/3/action/datastore_search'
resource_id = '49fa9ca0-f609-4ae3-a6f7-b97bd0945a3a'
query = 'GD.CE'
limit = 100  # Número de registros por página

# Função para obter o total de registros
def get_total_records():
    url = f'{base_url}?resource_id={resource_id}&q={query}&limit=1'
    print(f'Fetching total records from: {url}')  # Mensagem de depuração
    retries = 3
    delay = 2
    for attempt in range(retries):
        try:
            response = req.get(url)
            if response.status_code == 200:
                data = response.json()
                return data['result']['total']
            else:
                print(f'Erro na requisição: {response.status_code}. Tentativa {attempt + 1} de {retries}')
                time.sleep(delay)
        except Exception as e:
            print(f'Erro na requisição: {e}. Tentativa {attempt + 1} de {retries}')
            time.sleep(delay)
    raise Exception('Falha ao obter o total de registros.')

# Função para obter dados de uma URL
def fetch_data(offset):
    url = f'{base_url}?resource_id={resource_id}&q={query}&limit={limit}&offset={offset}'
    print(f'Fetching data from: {url}')  # Mensagem de depuração
    retries = 3
    delay = 2
    for attempt in range(retries):
        try:
            response = req.get(url)
            if response.status_code == 200:
                return response.json()
            else:
                print(f'Erro na requisição: {response.status_code}. Tentativa {attempt + 1} de {retries}')
                time.sleep(delay)
        except Exception as e:
            print(f'Erro na requisição: {e}. Tentativa {attempt + 1} de {retries}')
            time.sleep(delay)
    return None  # Retorna None se falhar após tentativas

# Função para buscar e processar dados
def fetch_all_data(total_records, limit):
    all_records = []
    offsets = range(0, total_records, limit)

    with ThreadPoolExecutor(max_workers=4) as executor:
        futures = {executor.submit(fetch_data, offset): offset for offset in offsets}

        for future in as_completed(futures):
            data = future.result()
            if data:
                all_records.extend(data['result']['records'])
            else:
                print('Falha ao obter dados, continuando com outras requisições.')

    return all_records

# Obter o total de registros
total_records = get_total_records()
print(f'Total de registros disponíveis: {total_records}')

# Obter todos os dados
all_records = fetch_all_data(total_records, limit)

# Exibir o número total de registros coletados
print(f'Total de registros coletados: {len(all_records)}')

# Opcional: salvar os dados em um arquivo JSON
with open('dados_distribuicao.json', 'w') as f:
    json.dump(all_records, f, indent=4)

print('Dados salvos em dados_distribuicao.json')


Fetching total records from: https://dadosabertos.aneel.gov.br/api/3/action/datastore_search?resource_id=49fa9ca0-f609-4ae3-a6f7-b97bd0945a3a&q=GD.CE&limit=1
Total de registros disponíveis: 93457
Fetching data from: https://dadosabertos.aneel.gov.br/api/3/action/datastore_search?resource_id=49fa9ca0-f609-4ae3-a6f7-b97bd0945a3a&q=GD.CE&limit=100&offset=0
Fetching data from: https://dadosabertos.aneel.gov.br/api/3/action/datastore_search?resource_id=49fa9ca0-f609-4ae3-a6f7-b97bd0945a3a&q=GD.CE&limit=100&offset=100
Fetching data from: https://dadosabertos.aneel.gov.br/api/3/action/datastore_search?resource_id=49fa9ca0-f609-4ae3-a6f7-b97bd0945a3a&q=GD.CE&limit=100&offset=200
Fetching data from: https://dadosabertos.aneel.gov.br/api/3/action/datastore_search?resource_id=49fa9ca0-f609-4ae3-a6f7-b97bd0945a3a&q=GD.CE&limit=100&offset=300
Fetching data from: https://dadosabertos.aneel.gov.br/api/3/action/datastore_search?resource_id=49fa9ca0-f609-4ae3-a6f7-b97bd0945a3a&q=GD.CE&limit=100&offset=

In [3]:
# Verificação de dados 

print('Total de registros:', len(all_records))

all_records[:2]
# tempo extração dia 11/09/2024: 00:06:41

Total de registros: 93457


[{'_id': 207854,
  'DatGeracaoConjuntoDados': '2024-09-11T00:00:00',
  'CodGeracaoDistribuida': 'GD.CE.000.004.477',
  'MdaAreaArranjo': '76,00',
  'MdaPotenciaInstalada': '12,00',
  'NomFabricanteModulo': 'TRINA SOLAR',
  'NomFabricanteInversor': 'SUN2000',
  'DatConexao': '2021-01-28T00:00:00',
  'MdaPotenciaModulos': '15,40',
  'MdaPotenciaInversores': '12,00',
  'QtdModulos': 38,
  'NomModeloModulo': 'TSM-DE15H(II)',
  'NomModeloInversor': '12KTL-M0',
  'rank': 0.057308756},
 {'_id': 207855,
  'DatGeracaoConjuntoDados': '2024-09-11T00:00:00',
  'CodGeracaoDistribuida': 'GD.CE.000.004.480',
  'MdaAreaArranjo': '19,80',
  'MdaPotenciaInstalada': '2,50',
  'NomFabricanteModulo': 'YINGLI SOLAR',
  'NomFabricanteInversor': 'SMA',
  'DatConexao': '2015-12-01T00:00:00',
  'MdaPotenciaModulos': '2,88',
  'MdaPotenciaInversores': '2,50',
  'QtdModulos': 12,
  'NomModeloModulo': ' ',
  'NomModeloInversor': ' ',
  'rank': 0.057308756}]